# Atmospheric Pressure Level Visualization (ERA-Interim)

This example visualizes ERA-Interim reanalysis data on pressure levels — a common format for atmospheric datasets. The data has a regular lat/lon grid with discrete pressure levels (200, 500, 850 hPa), making it ideal for pyvista-xarray's `RectilinearGrid`.

We demonstrate:
- 3D visualization of geopotential height across pressure levels
- Auto-detection of CF-compliant coordinates
- Wind vector visualization at individual levels
- Vertical exaggeration for atmospheric data

In [ ]:
import numpy as np
import pyvista as pv
import xarray as xr

import pvxarray  # noqa: F401

## Load ERA-Interim Data

The `eraint_uvz` tutorial dataset contains monthly-mean fields on three pressure levels:
- **z**: Geopotential height (m^2/s^2)
- **u**: Zonal (east-west) wind component (m/s)
- **v**: Meridional (north-south) wind component (m/s)

The coordinates have proper CF `units` attributes (`degrees_east`, `degrees_north`, `millibars`), so pyvista-xarray can auto-detect the axes:

In [ ]:
ds = xr.tutorial.load_dataset("eraint_uvz")

# Inspect the auto-detected axes
da = ds.z.isel(month=0)
print(f"Shape: {da.shape} (level x lat x lon)")
print(f"Detected axes: {da.pyvista.axes}")
print(f"Spatial coords: {da.pyvista.spatial_coords}")

## Single Pressure Level

Visualize geopotential height at 500 hPa (mid-troposphere). The coordinate auto-detection maps `longitude` to X and `latitude` to Y:

In [ ]:
z500 = ds.z.sel(level=500, month=1)

# Auto-detection works because coords have CF units attributes
z500.pyvista.plot(cpos="xy", cmap="RdYlBu_r")

## Multi-Level 3D Visualization

Visualize all three pressure levels as a 3D volume. The `level` coordinate (in millibars) becomes the Z axis. We use vertical exaggeration to make the pressure levels visible since atmospheric depth is tiny compared to horizontal extent:

In [ ]:
z_jan = ds.z.isel(month=0)

mesh = z_jan.pyvista.mesh(x="longitude", y="latitude", z="level")
print(f"Mesh type: {type(mesh).__name__}")
print(f"Points: {mesh.n_points}, Cells: {mesh.n_cells}")

pl = pv.Plotter()
pl.add_mesh(mesh, cmap="RdYlBu_r")
pl.set_scale(zscale=0.5)  # Exaggerate vertical
pl.show()

## Winter vs Summer Comparison

Compare the 500 hPa geopotential height between January (month=1) and July (month=7). The jet stream position and polar vortex strength change dramatically between seasons:

In [ ]:
pl = pv.Plotter(shape=(1, 2))

for i, (m, label) in enumerate([(1, "January"), (7, "July")]):
    pl.subplot(0, i)
    da = ds.z.sel(level=500, month=m)
    mesh = da.pyvista.mesh(x="longitude", y="latitude")
    pl.add_mesh(mesh, cmap="RdYlBu_r", clim=(4.8e4, 5.9e4))
    pl.add_text(f"{label} 500 hPa", font_size=10)
    pl.view_xy()

pl.link_views()
pl.show()

## Wind Speed at 200 hPa

Compute wind speed from u and v components at the 200 hPa level (near the tropopause) to visualize the jet stream:

In [ ]:
u200 = ds.u.sel(level=200, month=1)
v200 = ds.v.sel(level=200, month=1)
wind_speed = np.sqrt(u200**2 + v200**2)
wind_speed.name = "wind_speed"
wind_speed.attrs["units"] = "m/s"

wind_speed.pyvista.plot(
    x="longitude",
    y="latitude",
    cpos="xy",
    cmap="hot_r",
)